**1. Таблица tickets включает следующие поля:**

* date – дата продажи
* id – идентификатор чека
* manager – менеджер
* revenue – сумма чека

**Выберите некорректные конструкции запросов. Внимательно читайте варианты ответов: при каждой попытке список вариантов может меняться!**

* SELECT date, manager, AVG(revenue) as AvgRev FROM tickets GROUP BY date, manager WHERE AVG(revenue)>1000;
* SELECT COUNT(DISTINCT manager) FROM tickets;
* SELECT manager, SUM(revenue) as SumRev FROM tickets GROUP BY revenue;
* SELECT manager, id, MAX(revenue) as MaxRev FROM tickets GROUP BY manager HAVING MAX(revenue)>5000;
* SELECT manager, COUNT(id) as CountId FROM tickets GROUP BY manager HAVING CountId < 10;
* SELECT manager, SUM(revenue) as revenue FROM tickets GROUP BY manager;


**2. В чем заключается главное отличие HAVING от WHERE? Выберите верное утверждение или утверждения.**

* HAVING выполняется перед агрегацией/группировкой, WHERE – после
* HAVING можно использовать без агрегации
* WHERE можно использовать без агрегации
* **WHERE выполняется перед агрегацией/группировкой, HAVING – после**
* Ничем, они совершенно одинаковые!

In [32]:
import pandas as pd
import numpy as np
import pandahouse as ph
from datetime import date, datetime
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

**3. Продолжим работать с таблицей checks!**

**Посчитайте, сколько покупок приходится на каждого клиента, указав имя столбца как NumChecks. Результирующую таблицу с UserID, NumChecks отсортируйте по убыванию (DESC) нового столбца. Ограничение вывода – 10.**


In [14]:
query = """
SELECT 
    UserID,
    count(BuyDate) as NumChecks
FROM checks
Group by UserID
order by NumChecks DESC
LIMIT 10
"""
df = ph.read_clickhouse(query, connection=connection)
df

,UserID,NumChecks
0,12506794130929660079,276
1,640807012715027711,276
2,9851217944924628313,274
3,3220599270443999977,272
4,109604902333456192,270
5,17587339617847831868,270
6,8752522755161623524,269
7,6295159848944215572,267
8,5893749085932398221,266
9,10032596276912502803,266


**4. Давайте немного усложним задачу, и посчитаем ещё и сумму сделанных покупок в рублях (Rub).**

**Выведите в результирующую таблицу UserID клиентов, количество сделанных покупок (NumChecks) и сумму потраченных денег (Revenue). Результат отсортируйте по убыванию Revenue, оставив 10 записей.**


In [15]:
query = """
SELECT 
    UserID,
    count(BuyDate) as NumChecks,
    SUM(Rub) as Revenue
FROM checks
Group by UserID
order by Revenue DESC
LIMIT 10
"""
df = ph.read_clickhouse(query, connection=connection)
df

,UserID,NumChecks,Revenue
0,15605251414578189718,3,181146
1,16967914029594691558,10,139992
2,9613023063681218234,51,70128
3,11863769084622496656,32,63193
4,958354669642668056,49,60196
5,6911136753844832704,79,57721
6,10398072986532593332,21,57301
7,8208688783013614056,114,51605
8,5471859434812058036,82,46470
9,12791764976753443152,70,45662


**5. Теперь давайте попробуем с помощью группировки и агрегатных функций посмотреть динамику трат клиентов. Для этого нужно посчитать по дням минимальный, средний, и максимальный чек. Иными словами, необходимо сгруппироваться по датам, и использовать агрегатные функции MIN, AVG, MAX.**

**Выведите минимальный (MinCheck), максимальный (MaxCheck) и средний (AvgCheck) чек по дням. В результирующей таблице используйте сортировку по убыванию (DESC) столбца BuyDate. Как и на предыдущем шаге, ограничение на вывод равно 10.**


In [16]:
query = """
SELECT 
    BuyDate,
    Min(Rub) as MinCheck,
    Max(Rub) as MaxCheck,
    AVG(Rub) as AvgCheck
FROM checks
Group by BuyDate
order by BuyDate DESC
LIMIT 10
"""
df = ph.read_clickhouse(query, connection=connection)
df

,BuyDate,MinCheck,MaxCheck,AvgCheck
0,2019-10-04,0,3239,11.054391
1,2019-10-03,0,4898,10.357468
2,2019-10-02,0,3652,9.743485
3,2019-10-01,0,2928,9.813259
4,2019-09-30,0,4189,9.755777
5,2019-09-29,0,3213,11.740202
6,2019-09-28,0,4838,12.686164
7,2019-09-27,0,3232,11.156494
8,2019-09-26,0,7440,10.465333
9,2019-09-25,0,13906,10.394969


**6. Представим, что для проведения новой акции нам потребовалось выгрузить сегмент пользователей, которые приносят наибольшую выручку.**

**Найдите покупателей, сумма покупок которых превышает 10 000 рублей (Rub). Для этого сгруппируйте пользователей по UserID, затем примените нужную агрегатную функцию и назовите новую колонку Revenue. Отфильтруйте нужные значения, результат отсортируйте по убыванию UserID и выведите 10 строк.**


In [17]:
query = """
SELECT 
    UserID,
    sum(Rub) as Revenue
FROM checks
Group by UserID
HAVING Revenue > 10000
order by UserID DESC
LIMIT 10
"""
df = ph.read_clickhouse(query, connection=connection)
df

,UserID,Revenue
0,18266959643473967483,20395
1,18261532742663857859,13787
2,18105450744137216332,19858
3,17961878534766141140,17649
4,17907093951683732524,16556
5,17830166523985934832,11611
6,17789705480254581289,10834
7,17755116231546697738,26491
8,17717886684516945988,15055
9,17649746190216015646,22888


**7. В этой и последующих задачах мы будем работать с таблицей retail в ClickHouse, которая содержит данные о покупках.** 

**Задания выполняются в ClickHouse, в качестве ответа, как правило, нужно будет вписывать значения какого-либо поля из результирующей таблицы.**

Таблица включает следующие поля: 

* InvoiceNo — идентификатор покупки
* StockCode — идентификатор товара на складе
* Description — текстовое описание товара
* Quantity — количество товара
* InvoiceDate — дата покупки
* UnitPrice — цена товара
* CustomerID — идентификатор покупателя
* Country — страна

**Представим, что к вам пришел менеджер с проблемой: за последний месяц он наблюдает снижение выручки в некоторых регионах и хочет определить возможную причину этого снижения. Для начала можно сгруппировать данные о выручке по странам и найти среди них топ-5 стран по величине выручки, так как интереснее всего нам будет смотреть именно на данные этих стран.**

**Посчитайте выручку (Revenue) по странам и отсортируйте таблицу в порядке убывания выручки. В качестве ответа скопируйте название страны с самой высокой выручкой.  Обратите внимание, что в изначальной таблице указана цена одной единицы товара (UnitPrice) и количество купленного товара (Quantity), а не выручка, и для получения Revenue нужно будет использовать агрегирующую функцию Sum и оператор умножения *.**


Ответ: `United Kingdom`

```sql
SELECT 
    Country,
    SUM(UnitPrice * Quantity) as Revenue
FROM retail
GROUP BY Country
ORDER BY Revenue DESC
LIMIT 5
```

**8. В качестве более подробного описания выручки по странам можно посмотреть, каково среднее количество купленных товаров и средняя цена товара в покупках, совершенных в определенной стране.**

**Посчитайте среднее число купленных товаров по стране и среднюю цену товара, сгруппировав данные по странам и используя агрегирующую функцию Avg, и отсортируйте по убыванию средней цены товара (DESC).**

**Обратите внимание, что в данных встречаются строки с Description 'Manual', которые включают данные об удаленных из чека позициях. Для получения правильного ответа такие строки необходимо отфильтровать с использованием операторов WHERE и неравенства !=.**

**В качестве ответа впишите название страны, в которой самая высокая средняя цена за купленный товар.**

Ответ: `Canada`

```sql
 SELECT
    Country,
    avg(Quantity) AS avg_quantity,
    avg(UnitPrice) as avg_unit_price
FROM 
    retail    
Where Description != 'Manual'
GROUP BY 
    Country    
ORDER BY 
    avg_unit_price DESC
Limit 1
```

**9. Теперь посмотрим на динамику общей суммы выручки по месяцам.**

**Посчитайте выручку (Revenue) по месяцам (обратите внимание, что в таблице указана цена одной единицы товара и количество купленного товара), округлив InvoiceDate к началу месяца с помощью toStartOfMonth().**

**В качестве ответа укажите сумму выручки (Revenue) из первой строки результирующей таблицы, используя сортировку по убыванию (DESC) по столбцу Revenue. Не забывайте отфильтровать строки по условию Description != 'Manual'**

Ответ: `1130664.25`

```sql
 SELECT
    toStartOfMonth(InvoiceDate) as month,
    SUM(UnitPrice * Quantity) as Revenue
FROM 
    retail    
Where Description != 'Manual'
Group by month
order by Revenue desc
Limit 1
```

**10. Давайте посмотрим на динамику выручки от покупателей, которые в среднем покупают самые дорогие товары.  В качестве целевой метрики будем использовать среднюю цену купленного товара (UnitPrice), данные посмотрим за март 2011 года.**

**Посчитайте среднюю цену товара клиента (UnitPrice) с группировкой по id покупателя, используя агрегирующую функцию Avg, затем выведите строки с покупками тех покупателей, у которых самая высокая средняя цена купленного товара в марте. Не забывайте отфильтровать строки по условию Description != 'Manual'. В качестве ответа впишите CustomerID покупателя с самой высокой средней ценой купленного товара.**

Ответ: `13452`

```sql
 SELECT
    CustomerID,
    avg(UnitPrice) as Revenue
FROM 
    retail    
Where Description != 'Manual' and toStartOfMonth(InvoiceDate) = '2011-03-01'
Group by CustomerID
order by Revenue desc
Limit 1
```

**11. Давайте попробуем ответить на следующий вопрос: "Как изменилось среднее, минимальное и максимальное количество купленного товара в стране с наибольшей выручкой в течение последних месяцев?"**

**Для этого нужно сгруппировать данные по странам и месяцам, посчитать среднее (Avg), минимальное (Min) и максимальное (Max) количество купленных товаров (Quantity), и отсортировать данные по месяцу, применить фильтр по стране (нам нужны данные из страны с наибольшей выручкой из задания 8 (step 8)). Если обратить внимание на полученное значение минимального количества товаров, то станет понятно, что там есть отрицательные значения, которые следует отфильтровать (также не забывайте отфильтровывать значения с Description != 'Manual').**

**В качестве ответа скопируйте наибольшее историческое значение среднего числа купленных товаров для этой страны.**

Ответ: `16.569958371877892`

```sql
 SELECT
    toStartOfMonth(InvoiceDate) as month,
    avg(Quantity) as avg_Quantity,
    min(Quantity) as min_Quantity,
    max(Quantity) as max_Quantity
FROM 
    retail    
Where Description != 'Manual'and Country = 'United Kingdom' and Quantity>0
Group by  month
order by avg_Quantity desc 
Limit 1
```